<a href="https://colab.research.google.com/github/sjnaj/DeepLearning/blob/master/Basic/layersAndBlocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[-0.0708,  0.1940, -0.1375, -0.1126, -0.0094, -0.2533, -0.0626,  0.0696,
         -0.2112,  0.1389],
        [-0.0099,  0.0608, -0.0250, -0.0163,  0.0969, -0.3051, -0.1870, -0.1103,
         -0.1439,  0.1661]], grad_fn=<AddmmBackward0>)

自定义块

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):#在nn.Module中映射到__self__上
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [4]:
net=MLP()
net(X)

tensor([[-0.0938, -0.3164,  0.1827,  0.0036,  0.0163, -0.1917, -0.2530,  0.0999,
          0.0322, -0.0096],
        [-0.0999, -0.1062,  0.1479, -0.0119,  0.1503,  0.0283, -0.1485,  0.0617,
          0.0884, -0.1171]], grad_fn=<AddmmBackward0>)

自定义顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X


In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.3066, -0.0166, -0.0799, -0.0569,  0.2381,  0.0321,  0.0371,  0.0777,
          0.0003, -0.0135],
        [-0.0912, -0.0402, -0.0250,  0.0942,  0.1888, -0.0461,  0.0055,  0.1226,
          0.1062, -0.1039]], grad_fn=<AddmmBackward0>)

灵活的forward

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)#False当作标量，否则当作变量
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2742, grad_fn=<SumBackward0>)

混搭各种块

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.2437, grad_fn=<SumBackward0>)